# Interpolate Calanus
Interpolate (univariate) the observations of _Calanus finmarchicus_ and _Calanus helgolandicus_.

The fields are prepared over a region covering the Northeast Atlantic.      
Different resolutions are tested: 0.5° X 0.5°, 1° X 1° and 2° X 2°.

In order to modify the directories, the spatial resolution, edit the configuration file [config.jl](../src/config.jl).

In [1]:
using Dates
using NCDatasets
using PyPlot
using DIVAnd
using Statistics
const plt=PyPlot
using PyCall
using PyPlot
include("../src/InterpCalanus.jl")

Main.InterpCalanus

In [2]:
include("../src/config.jl")

[ Info: Workig at resoluton 1.0° X 1.0°
[ Info: Observation file already downloaded
[ Info: Bathymetry file already downloaded
[ Info: Temperature file already downloaded


"#456A30"

## Install `Cartopy`
Set `usecartopy` to `false` if you don't want to use it.

In [3]:
if usecartopy
    @info("Will try to use Cartopy for the plots")
    cartopy = PyCall.pyimport("cartopy")
    ccrs = PyCall.pyimport("cartopy.crs")
    cfeature = PyCall.pyimport("cartopy.feature")
    coast = cfeature.GSHHSFeature(scale="full");
    cartopyticker = PyCall.pyimport("cartopy.mpl.ticker")
    lon_formatter = cartopyticker.LongitudeFormatter()
    lat_formatter = cartopyticker.LatitudeFormatter()
    mainproj = ccrs.Mercator(central_longitude=0.5*(domain[1] + domain[2]), 
        min_latitude=domain[3], max_latitude=domain[4])
    datacrs = ccrs.PlateCarree();
else
    @info("Won't use cartopy")
end

[ Info: Won't use cartopy


## User inputs
### Type of analysis to be performed
Set the values (`true` or `false`) for the parameters `run_year` and `run_month = true in `config.jl`.

## Grid configuration

In [4]:
longrid = domain[1]:dlon:domain[2]
latgrid = domain[3]:dlat:domain[4]
@info("Size of the grid: $(length(longrid)) X $(length(latgrid))")

[ Info: Size of the grid: 33 X 26


### Compute metrics

In [5]:
_, (pm, pn), (xi, yi) = DIVAnd_rectdom(longrid, latgrid);
@info(size(xi));

[ Info: (33, 26)


### Prepare land-sea mask
Using depth ≃ 5 meters

In [6]:
bx, by, b = DIVAnd.load_bath(bathyfile, true, longrid, latgrid)
@show size(b);

_, _, mask = DIVAnd.load_mask(bathyfile, true, longrid, latgrid, 5.0);
# Remove Mediterranean Sea
maskindex = findall((xi .>= 0.) .& (yi .<= 47.))
mask[maskindex] .= 0.;

size(b) = (33, 26)


## Data reading
### Read the CSV files and separate by columns

In [7]:
@time lon, lat, dates, calanus_finmarchicus, calanus_helgolandicus = InterpCalanus.read_data_calanus(datafile);

  1.193117 seconds (5.66 M allocations: 191.113 MiB, 5.42% gc time, 67.45% compilation time)


[ Info: (-20.447, 11.703)
[ Info: (41.2567, 66.808)


## Analysis
### 1. Basic analysis: all data together
- The objective is to show the results of the analysis with short, medium and large correlation lengths.
- The background field is set to zero: no observations means that there is 'nothing'.

In [8]:
L = 2.5
epsilon2 = 5.

5.0

In [9]:
@time f_finmarchicus, s = DIVAndrun(mask, (pm,pn), (xi,yi), (lon, lat), 
    calanus_finmarchicus .- 0. * mean(calanus_finmarchicus), (L, L), epsilon2);
@time f_helgolandicus, s = DIVAndrun(mask, (pm,pn), (xi,yi), (lon, lat), 
    calanus_helgolandicus .- 0. * mean(calanus_helgolandicus), (L, L), epsilon2);

 21.444272 seconds (47.06 M allocations: 2.465 GiB, 4.46% gc time, 98.35% compilation time)
  0.286181 seconds (2.05 M allocations: 148.363 MiB)


### 2. Long correlation length

In [10]:
Llong = 100
epsilon2 = 5.
@time f_finmarchicus, s = DIVAndrun(mask, (pm,pn), (xi,yi), (lon, lat), 
    calanus_finmarchicus .- 0. * mean(calanus_finmarchicus), (Llong, Llong), epsilon2);
@time f_helgolandicus, s = DIVAndrun(mask, (pm,pn), (xi,yi), (lon, lat), 
    calanus_helgolandicus .- 0. * mean(calanus_helgolandicus), (Llong, Llong), epsilon2);

if makeplot
    figname = joinpath(figdir, "analysis_calanus_finmarchicus_large.jpg")
    PlottingCalanus.make_plot_analysis(longrid, latgrid, f_finmarchicus, 
                       "Calanus finmarchicus analysis (L = $(Llong), eps2 = $(epsilon2))", figname,
                       mainproj)
end

  2.390701 seconds (6.35 M allocations: 361.632 MiB, 5.06% gc time, 83.65% compilation time)
  0.314608 seconds (2.05 M allocations: 148.363 MiB)


### 3. Short correlation length

In [11]:
Lshort = 0.5
epsilon2 = 5.
@time f_finmarchicus, s = DIVAndrun(mask, (pm,pn), (xi,yi), (lon, lat), 
    calanus_finmarchicus .- 0. * mean(calanus_finmarchicus), (Lshort, Lshort), epsilon2);
@time f_helgolandicus, s = DIVAndrun(mask, (pm,pn), (xi,yi), (lon, lat), 
    calanus_helgolandicus .- 0. * mean(calanus_helgolandicus), (Lshort, Lshort), epsilon2);

if makeplot
    figname = joinpath(figdir, "analysis_calanus_finmarchicus_short.jpg")
    PlottingCalanus.make_plot_analysis(longrid, latgrid, f_finmarchicus, 
                       "Calanus finmarchicus analysis (L = $(Lshort), eps2 = $(epsilon2))", figname,
                       mainproj)
end

  0.371461 seconds (2.05 M allocations: 148.292 MiB, 7.92% gc time)
  0.352493 seconds (2.05 M allocations: 148.292 MiB, 7.82% gc time)


## Time analysis
### Yearly
In these yearly analysis all the data are taken, whatever the month of the observation.     

#### Create the (empty) netCDF files:

In [12]:
resfileyear_combined = joinpath(resdir, "calanus_finmarchicus_helgolandicus_year.nc")

ntimes = (1 - yearmin + yearmax)
InterpCalanus.create_nc_results_time(resfileyear_combined, longrid, latgrid, ntimes,  
                      ["Calanus_finmarchicus", "Calanus_helgolandicus"]; valex=-999.9, 
                      varname="abundance",
                      long_name="Interpolated abundance",
                      domain=domain,
                      aphiaID=[Int32(104464), Int32(104466)], L, epsilon2)

21-element Vector{Char}:
 'C': ASCII/Unicode U+0043 (category Lu: Letter, uppercase)
 'a': ASCII/Unicode U+0061 (category Ll: Letter, lowercase)
 'l': ASCII/Unicode U+006C (category Ll: Letter, lowercase)
 'a': ASCII/Unicode U+0061 (category Ll: Letter, lowercase)
 'n': ASCII/Unicode U+006E (category Ll: Letter, lowercase)
 'u': ASCII/Unicode U+0075 (category Ll: Letter, lowercase)
 's': ASCII/Unicode U+0073 (category Ll: Letter, lowercase)
 '_': ASCII/Unicode U+005F (category Pc: Punctuation, connector)
 'h': ASCII/Unicode U+0068 (category Ll: Letter, lowercase)
 'e': ASCII/Unicode U+0065 (category Ll: Letter, lowercase)
 'l': ASCII/Unicode U+006C (category Ll: Letter, lowercase)
 'g': ASCII/Unicode U+0067 (category Ll: Letter, lowercase)
 'o': ASCII/Unicode U+006F (category Ll: Letter, lowercase)
 'l': ASCII/Unicode U+006C (category Ll: Letter, lowercase)
 'a': ASCII/Unicode U+0061 (category Ll: Letter, lowercase)
 'n': ASCII/Unicode U+006E (category Ll: Letter, lowercase)
 'd': ASCI

#### Loop on the years

In [13]:
if run_year 
    for (iii, yyyy) in enumerate(yearmin:yearmax)
        goodyear = findall(Dates.year.(dates) .== yyyy)
        @info("Number of observations for year $(yyyy): $(length(goodyear))")

        # Analysis
        f_finmarchicus, s_fin = DIVAndrun(mask, (pm,pn), (xi,yi), (lon[goodyear], lat[goodyear]), 
            calanus_finmarchicus[goodyear] .- 0. * mean(calanus_finmarchicus[goodyear]), (L, L), epsilon2);
        f_helgolandicus, s_hel = DIVAndrun(mask, (pm,pn), (xi,yi), (lon[goodyear], lat[goodyear]), 
            calanus_helgolandicus[goodyear] .- 0. * mean(calanus_helgolandicus[goodyear]), (L, L), epsilon2);
        
        # Error field
        cpme = DIVAnd_cpme(mask, (pm, pn), (xi, yi), (lon[goodyear], lat[goodyear]), 
            calanus_finmarchicus[goodyear], L, epsilon2);
        
        #cpme_helgo = DIVAnd_cpme(mask, (pm, pn), (xi, yi), (lon[goodyear], lat[goodyear]), 
        #    calanus_helgolandicus[goodyear], L, epsilon2);
        
        if makeplot
            # Figures
            figname = joinpath(figdir, "analysis_calanus_finmarchicus_$(yyyy).jpg")
            PlottingCalanus.make_plot_analysis(longrid, latgrid, f_finmarchicus .+ 0. * mean(calanus_finmarchicus[goodyear]), 
                "Calanus finmarchicus analysis ($(yyyy))", figname, mainproj)

            figname = joinpath(figdir, "analysis_calanus_helgolandicus_$(yyyy).jpg")
            PlottingCalanus.make_plot_analysis(longrid, latgrid, f_helgolandicus .+ 0. * mean(calanus_helgolandicus[goodyear]), 
                "Calanus helgolandicus analysis ($(yyyy))", figname, mainproj)

            figname = joinpath(figdir, "error_calanus_finmarchicus_$(yyyy).jpg")
            PlottingCalanus.make_plot_error(longrid, latgrid, cpme, lon[goodyear], lat[goodyear], 
                "Relative error field", figname, mainproj)
        end
        
        #figname = joinpath(figdir, "error_calanus_helgolandicus_$(yyyy).jpg")
        #PlottingCalanus.make_plot_error(longrid, latgrid, cpme_helgo, lon[goodyear], lat[goodyear], 
        #    "Relative error field", figname, mainproj)
        
        # write single netCDF files
        #InterpCalanus.create_nc_results(joinpath(resdirnc, "calanus_finmarchicus_$(yyyy).nc"), 
        #              longrid, latgrid, (Dates.Date(yyyy, 1, 1) - Dates.Date(1950, 1, 1)).value, 
        #              f_finmarchicus .+ 0. * mean(calanus_finmarchicus[goodyear]), L, epsilon2,
        #              "Calanus finmarchicus"; valex=-999.9, 
        #              varname="abundance",
        #              long_name="Abundance of Calanus finmarchicus",
        #              domain=domain,
        #              aphiaID=Int32(104464)
        #           )
        
        #InterpCalanus.write_nc_error(joinpath(resdirnc, "calanus_finmarchicus_$(yyyy).nc"), cpme)

        #InterpCalanus.create_nc_results(joinpath(resdirnc, "calanus_helgolandicus_$(yyyy).nc"), 
        #          longrid, latgrid, (Dates.Date(yyyy, 1, 1) - Dates.Date(1950, 1, 1)).value, 
        #          f_helgolandicus .+ 0. * mean(calanus_helgolandicus[goodyear]), L, epsilon2, 
        #          "Calanus helgolandicus"; valex=-999.9, 
        #          varname="abundance",
        #          long_name="Abundance of Calanus helgolandicus",
        #          domain=domain,
        #          aphiaID=Int32(104466)
        #       )
        
        #InterpCalanus.write_nc_error(joinpath(resdirnc, "calanus_helgolandicus_$(yyyy).nc"), cpme)
        
        # Write common netCDF files
        Dataset(resfileyear_combined, "a") do ds
            ds["time"][iii] = (Dates.Date(yyyy, 1, 1) - Dates.Date(1950, 1, 1)).value
            ds["error"][:,:,iii,1] = cpme
            ds["error"][:,:,iii,2] = cpme
            ds["abundance"][:,:,iii,1] = f_finmarchicus
            ds["abundance"][:,:,iii,2] = f_helgolandicus
        end

       
        # geoTIFF files
        #InterpCalanus.write_geotiff(joinpath(resdirtif, "calanus_finmarchicus_$(yyyy).tif"), f_finmarchicus, domain)
        #InterpCalanus.write_geotiff(joinpath(resdirtif, "calanus_helgolandicus_$(yyyy).tif"), f_helgolandicus, domain)
        #InterpCalanus.write_geotiff(joinpath(resdirtif, "calanus_finmarchicus_error_$(yyyy).tif"), cpme, domain)
        #InterpCalanus.write_geotiff(joinpath(resdir, "calanus_helgolandicus_error_$(yyyy).tif"), cpme_helgo, dom)

    end
end

[ Info: Number of observations for year 1959: 2364
[ Info: Number of observations for year 1960: 2321
[ Info: Number of observations for year 1961: 2382
[ Info: Number of observations for year 1962: 2564
[ Info: Number of observations for year 1963: 2465
[ Info: Number of observations for year 1964: 2556
[ Info: Number of observations for year 1965: 2777
[ Info: Number of observations for year 1966: 2422
[ Info: Number of observations for year 1967: 2679
[ Info: Number of observations for year 1968: 2489
[ Info: Number of observations for year 1969: 2863
[ Info: Number of observations for year 1970: 2701
[ Info: Number of observations for year 1971: 2964
[ Info: Number of observations for year 1972: 2808
[ Info: Number of observations for year 1973: 2788
[ Info: Number of observations for year 1974: 2600
[ Info: Number of observations for year 1975: 2490
[ Info: Number of observations for year 1976: 2361
[ Info: Number of observations for year 1977: 2267
[ Info: Number of observations 

### Monthly

In [50]:
resfilemonth_combined = joinpath(resdir, "calanus_finmarchicus_helgolandicus_year_month.nc")

InterpCalanus.create_nc_results_time(resfilemonth_combined, longrid, latgrid,  
                      ["Calanus_finmarchicus", "Calanus_helgolandicus"]; valex=-999.9, 
                      varname="abundance",
                      long_name="Abundance",
                      domain=domain,
                      aphiaID=[Int32(104464), Int32(104466)], L, epsilon2);

In [51]:
if run_month
    iii = 0
    for yyyy in 1959:1961

        for m in 1:12

            iii += 1

            mm = lpad(string(m), 2, '0')
            # Select the good observations
            goodtime = findall( (Dates.year.(dates) .== yyyy) .& (Dates.month.(dates) .== m) )
            @info("Found $(length(goodtime)) dates points for year $(yyyy) | month $(mm)")

            if length(goodtime) > 0

                f_finmarchicus, s = DIVAndrun(mask, (pm,pn), (xi,yi), (lon[goodtime], lat[goodtime]), 
                    calanus_finmarchicus[goodtime] .- 0. * mean(calanus_finmarchicus[goodtime]), (L, L), epsilon2);
                f_helgolandicus, s = DIVAndrun(mask, (pm,pn), (xi,yi), (lon[goodtime], lat[goodtime]), 
                    calanus_helgolandicus[goodtime] .- 0. * mean(calanus_helgolandicus[goodtime]), (L, L), epsilon2);

                # Error field
                cpme_fin = DIVAnd_cpme(mask, (pm, pn), (xi, yi), (lon[goodtime], lat[goodtime]), 
                    calanus_finmarchicus[goodtime], L, epsilon2);
                cpme_hel = DIVAnd_cpme(mask, (pm, pn), (xi, yi), (lon[goodtime], lat[goodtime]), 
                    calanus_helgolandicus[goodtime], L, epsilon2);

                if makeplot
                    figname = joinpath(figdir, "analysis_calanus_finmarchicus_$(yyyy)$(mm).jpg")         
                    PlottingCalanus.make_plot_analysis(longrid, latgrid, f_finmarchicus, 
                    "Calanus finmarchicus analysis ($(Dates.monthname(m)) $(yyyy))", figname, mainproj)

                    figname = joinpath(figdir, "analysis_calanus_helgolandicus_$(yyyy)$(mm).jpg")
                    #make_plot_analysis(longrid, latgrid, f_helgolandicus .+ mean(calanus_helgolandicus[goodtime]), 
                    #    "Calanus finmarchicus analysis ($(Dates.monthname(m)) $(yyyy))", figname)
                end

                Dataset(resfilemonth_combined, "a") do ds
                    ds["time"][iii] = (Dates.Date(yyyy, m, 1) - Dates.Date(1950, 1, 1)).value
                    ds["error"][:,:,iii,1] = cpme_fin
                    ds["error"][:,:,iii,2] = cpme_hel
                    ds["abundance"][:,:,iii,1] = f_finmarchicus
                    ds["abundance"][:,:,iii,2] = f_helgolandicus
                end

            else
                @warn("No data to perform analysis")
            end

        end

    end
end

[ Info: Found 141 dates points for year 1959 | month 01
[ Info: Found 144 dates points for year 1959 | month 02
[ Info: Found 247 dates points for year 1959 | month 03
[ Info: Found 124 dates points for year 1959 | month 04
[ Info: Found 202 dates points for year 1959 | month 05
[ Info: Found 227 dates points for year 1959 | month 06
[ Info: Found 216 dates points for year 1959 | month 07
[ Info: Found 220 dates points for year 1959 | month 08
[ Info: Found 239 dates points for year 1959 | month 09
[ Info: Found 270 dates points for year 1959 | month 10
[ Info: Found 183 dates points for year 1959 | month 11
[ Info: Found 151 dates points for year 1959 | month 12
[ Info: Found 160 dates points for year 1960 | month 01
[ Info: Found 186 dates points for year 1960 | month 02
[ Info: Found 197 dates points for year 1960 | month 03
[ Info: Found 196 dates points for year 1960 | month 04
[ Info: Found 245 dates points for year 1960 | month 05
[ Info: Found 179 dates points for year 1960 | m

In [52]:
@info("Results writtin in directory $(resdir)");

[ Info: Results writtin in directory ../results/multivariate/1deg_test/


## Plots
If `cartopy` could not be installed properly in Julia, you can use the Python notebook [plot_results_map_python](plot_results_map_python.ipynb) to create the figures using the netCDF files.